# SNAPI Demo
This notebook showcases very basic functionality of SNAPI, including querying databases, saving/loading files and manipulating Transient objects.

## (1) Creating a Transient object for 2023ixf.
Here, we will create a mostly empty Transient object for 2023ixf. We will add information through queries.

In [ ]:
from snapi import Transient

ixf_transient = Transient(iid="2023ixf")
print(ixf_transient.id)
print(ixf_transient.coordinates)

Next, we'll call TNS, ALeRCE, and ANTARES to add information based on the IAU name.

In [ ]:
from snapi.query_agents import TNSQueryAgent, ALeRCEQueryAgent, ANTARESQueryAgent

tns_query_agent = TNSQueryAgent()
alerce_query_agent = ALeRCEQueryAgent()
antares_query_agent = ANTARESQueryAgent()

for agent in [tns_query_agent, alerce_query_agent, antares_query_agent]:
    query_results, _ = agent.query_transient(ixf_transient)
    for query_result in query_results:
        ixf_transient.ingest_query_info(query_result.to_dict())
    print(ixf_transient.internal_names)
    print(len(ixf_transient.photometry))

In [ ]:
# see all unique filters
print(len(ixf_transient.photometry))
for lc in ixf_transient.photometry.light_curves:
    print(lc.filter)

In [ ]:
# see all extracted information
print(ixf_transient.coordinates)
print(ixf_transient.redshift)
print(ixf_transient.spec_class)  # TODO: add spec class extraction

In [ ]:
# plot all photometry
import matplotlib.pyplot as plt
import numpy as np
from snapi import Formatter

formatter = Formatter()
fig, ax = plt.subplots(figsize=(8, 6))
ixf_transient.photometry.plot(ax, formatter=formatter)
formatter.make_plot_pretty(ax)
formatter.add_legend(ax, ncols=2)
plt.show()

In [ ]:
# can also plot in flux space
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 6))
ixf_transient.photometry.plot(ax, mags=False)
formatter.add_legend(ax, ncols=2)
formatter.make_plot_pretty(ax)
plt.show()

In [ ]:
from snapi import Formatter
import matplotlib.pyplot as plt

# can also plot each LC individually
formatter = Formatter()
fig, axes = plt.subplots(4, 2, figsize=(10, 14))
for i, lc in enumerate(ixf_transient.photometry.light_curves):
    ax = axes.flatten()[i]
    lc.plot(ax, formatter)
    ax.legend(loc="best")
    # formatter.make_plot_pretty(ax)
plt.tight_layout()
plt.show()

In [ ]:
# let's save this object for later
ixf_transient.save(filename="data/ixf_transient.hdf5")

## Example 2: Querying by RA/dec
Whereas the previous example queries by IAU name, we instead consider an example where we query from RA/dec coordinates. For this example, we use a dataset of DECAM DDF "likely-real" candidates: https://arxiv.org/abs/2211.09202 

In [ ]:
# import DECAM data
import pandas as pd
import numpy as np

decam_fn = "data/candidates.dat"
decam_df = pd.read_table(
    decam_fn,
    sep=r"\s+",
    comment="#",
    names=["field", "id", "ra", "dec", "n_obs", "mean_rb"],
    usecols=np.arange(6),
)
print(decam_df.head())

There are two sets of fields in the DECAM deep-drilling fields: 3 fields that overlap with COSMOS (declination ~1-4 degrees), and 2 fields at lower declination (-45 -> -42 deg). Let's display them:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# plot COSMOS ra/dec distribution
cosmos_ddf = decam_df[decam_df["field"] == "COSMOS"]
l = ax[0].scatter(
    cosmos_ddf["ra"], cosmos_ddf["dec"], c=cosmos_ddf["mean_rb"], cmap="viridis", vmax=1.0, s=10
)
ax[0].set_title("COSMOS fields")

# plot non-COSMOS fields
non_cosmos_ddf = decam_df[decam_df["field"] != "COSMOS"]
l2 = ax[1].scatter(
    non_cosmos_ddf["ra"], non_cosmos_ddf["dec"], c=non_cosmos_ddf["mean_rb"], cmap="viridis", vmax=1.0, s=10
)
ax[1].set_title("Non-COSMOS fields")
fig.colorbar(label="Mean real-bogus score", mappable=l2)

ax[0].set_xlabel("RA")
ax[1].set_xlabel("RA")
ax[0].set_ylabel("Dec")

plt.show()

As we can see, all events in this dataset have a mean real-bogus score above 0.4, which is the "probably-real" threshhold according to the paper, so we keep all events for cross-matching.

In [ ]:
# now colorbar by number of observations

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# plot COSMOS ra/dec distribution
cosmos_ddf = decam_df[decam_df["field"] == "COSMOS"]
l = ax[0].scatter(cosmos_ddf["ra"], cosmos_ddf["dec"], c=cosmos_ddf["n_obs"], cmap="viridis", vmax=100, s=10)
ax[0].set_title("COSMOS fields")

# plot non-COSMOS fields
non_cosmos_ddf = decam_df[decam_df["field"] != "COSMOS"]
l2 = ax[1].scatter(
    non_cosmos_ddf["ra"], non_cosmos_ddf["dec"], c=non_cosmos_ddf["n_obs"], cmap="viridis", vmax=100, s=10
)
ax[1].set_title("Non-COSMOS fields")
fig.colorbar(label="Number of observations", mappable=l2)

ax[0].set_xlabel("RA")
ax[1].set_xlabel("RA")
ax[0].set_ylabel("Dec")

plt.show()

Many events (those in yellow) have over 100 observations; they are probably AGN or very long-duration TDEs/SNe.

Because northern sky telescopes like ZTF and Pan-STARRS have a minimum declination > ~-30 degrees, we expect potential cross-matches in TNS with only the COSMOS fields. Let's use SNAPI to make this cross-matching simple:

In [ ]:
import astropy.units as u
from snapi import Transient
from snapi.query_agents import TNSQueryAgent

tns_query_agent = TNSQueryAgent()
cross_matched_transients = set()


for i, row in decam_df.iterrows():
    if i % 500 == 0:
        print(f"Processing transient {i}/{len(decam_df)}")
    decam_transient = Transient(
        iid=row["id"],
        ra=row["ra"] * u.deg,  # pylint: disable=no-member
        dec=row["dec"] * u.deg,  # pylint: disable=no-member
    )

    # query TNS
    query_results, _ = tns_query_agent.query_transient(decam_transient, local=True)
    for query_result in query_results:
        decam_transient.ingest_query_info(query_result.to_dict())

    # check if IAU name found
    if decam_transient.internal_names:  # check if non-empty
        print(f"Matched {row['id']} with TNS object {decam_transient.spec_class} {decam_transient.id}")
        cross_matched_transients.add(decam_transient)

We see we've successfully crossmatched a handful of DDF events with TNS! Let's save the Transient objects so we don't have to rerun that loop every time. Luckily, SNAPI has extremely straightforward save/load functionality:

In [ ]:
# save matching transients

import pandas as pd
import glob

for transient in cross_matched_transients:
    transient.save(f"data/ddf_transient_{transient.id}.hdf5")

Now that we've reduced the dataset down to a small number, let's run a full query loop on each event:

In [ ]:
import glob
import matplotlib.pyplot as plt
from snapi.query_agents import TNSQueryAgent, ALeRCEQueryAgent, ANTARESQueryAgent
from snapi import Transient, Formatter

tns_agent = TNSQueryAgent()
alerce_agent = ALeRCEQueryAgent()
antares_agent = ANTARESQueryAgent()
formatter = Formatter()

# ensure we can load the transients
for f in glob.glob("data/ddf_transient_*.hdf5"):
    transient = Transient.load(f)

    # small internal names fix
    internal_names = set(transient.internal_names)
    if "nan" in transient.internal_names:
        internal_names.remove("nan")
    transient.internal_names = internal_names

    for agent in [tns_agent, antares_agent]:
        query_results, _ = agent.query_transient(transient)
        for query_result in query_results:
            transient.ingest_query_info(query_result.to_dict())

    fig, ax = plt.subplots(figsize=(6, 4))
    transient.photometry.plot(ax)
    ax.set_title(f"{transient.id} ({", ".join(transient.internal_names)})")
    formatter.make_plot_pretty(ax)
    formatter.add_legend(ax, ncols=2)
    plt.show()

    # now re-save
    transient.save(f"data/ddf_transient2_{transient.id}.hdf5")

2019tzk looks like an AGN, and has significant ZTF data. Let's overlay our DECAM data by adding it to the Transient object and re-plotting.

In [ ]:
import pandas as pd
import numpy as np
from astropy.time import Time
from astropy.timeseries import TimeSeries
from astropy import units as u
import matplotlib.pyplot as plt

from snapi import LightCurve, Filter, Formatter, Transient
import os

transient_2019tzk = Transient.load("data/ddf_transient2_2019tzk.hdf5")
formatter = Formatter()

observations_df = pd.read_csv(
    "data/DC21fyx.csv",
    usecols=[4, 5, 6, 7, 8],
    header=0,
)
print(observations_df.head())

for band in np.unique(observations_df["filter"]):
    band_df = observations_df[observations_df["filter"] == band]
    filt = Filter(
        instrument="DECam",
        band=band,
        center=np.nan * u.AA,
    )
    time_mjds = Time(band_df["meanmjd"].values, format="mjd")
    lc = LightCurve(
        times=time_mjds,
        fluxes=band_df["flux"].values,
        flux_errs=band_df["fluxerr"].values,
        zpts=band_df["magzp"].values,
        filt=filt,
    )
    transient_2019tzk.photometry.add_lightcurve(lc)

fig, ax = plt.subplots(figsize=(12, 8))
transient_2019tzk.photometry.plot(ax)
ax.set_title("2019tzk (with DECam data)")
formatter.make_plot_pretty(ax)
formatter.add_legend(ax, ncols=2)
plt.show()

print(f"Total observations: {len(transient_2019tzk.photometry.time_series)}")


transient_2019tzk_path = os.path.abspath("data/ddf_transientdecam_2019tzk.hdf5")
transient_2019tzk.save(transient_2019tzk_path)

We see here that the DECam and ZTF data align very well, capturing the same stochastic behavior of the AGN. However, we note that above, there were TWO DECam events that mapped to 2019tzk, the other being DC21crhk. Let's add that data in as well:

In [ ]:
import pandas as pd
import numpy as np
from astropy.time import Time
from astropy.timeseries import TimeSeries
from astropy import units as u
import matplotlib.pyplot as plt

from snapi import LightCurve, Filter, Formatter, Transient
import os

transient_2019tzk = Transient.load("data/ddf_transientdecam_2019tzk.hdf5")
formatter = Formatter()

observations_df = pd.read_table(
    "data/DC21crhk.csv",
    usecols=[5, 6, 7, 8],
    names=["meanmjd", "filter", "mag", "magerr"],
    sep=r"\s+",
)

for band in np.unique(observations_df["filter"]):
    band_df = observations_df[observations_df["filter"] == band]
    filt = Filter(
        instrument="DECam",
        band=band,
        center=np.nan * u.AA,
    )
    time_mjds = Time(band_df["meanmjd"].values, format="mjd")
    lc = LightCurve(times=time_mjds, mags=band_df["mag"].values, mag_errs=band_df["magerr"].values, filt=filt)
    transient_2019tzk.photometry.add_lightcurve(lc)

fig, ax = plt.subplots(figsize=(12, 8))
transient_2019tzk.photometry.plot(ax)
ax.set_title("2019tzk (with more DECam data)")
formatter.make_plot_pretty(ax)
formatter.add_legend(ax, ncols=2)
plt.show()

print(f"Total observations: {len(transient_2019tzk.photometry.time_series)}")
transient_2019tzk_path = os.path.abspath("data/ddf_transientdecam_2019tzk.hdf5")
transient_2019tzk.save(transient_2019tzk_path)

Only 11 observations were added, so not much difference is noted!

## Example 3: Modifying light curves for ML applications

After colating all information about a transient event, we may still need to restructure that information so it can be used for machine learning applications. For this tutorial, we will focus on formatting light curves for RNNs and convolutional neural networks, as well as light curve augmentation for better training performance.

In [ ]:
# for this example, we'll be using ZTF photometry from 2023ixf.
from snapi import Transient
import matplotlib.pyplot as plt
from snapi import Formatter

formatter = Formatter()
ixf_transient = Transient.load("data/ixf_transient.hdf5")
ixf_photometry = ixf_transient.photometry
ztf_photometry = ixf_photometry.filter({"ZTF_g", "ZTF_r"})

fig, ax = plt.subplots(figsize=(8, 6))
ztf_photometry.plot(ax, formatter=formatter)
formatter.make_plot_pretty(ax)
formatter.add_legend(ax, ncols=2)
plt.show()

In [ ]:
lcs = ztf_photometry.light_curves  # returns a copy of _light_curves
for lc in lcs:
    lc.phase()
ztf_photometry = Photometry(lcs)

fig, ax = plt.subplots(figsize=(8, 6))
ztf_photometry.plot(ax, formatter=formatter)
formatter.make_plot_pretty(ax)
formatter.add_legend(ax, ncols=2)
plt.show()

REST OF DEMO:
Basic functionality:
- retrieve detections vs non-detections (maybe return LC not time series)
- add_observation and remove_observation
- add LC and remove LC
- filter photometry by filter

ML functionality:
- phase photometry (implement peak and phase functions)
- show photometry tiling
- show photometry dense_lc
- show photometry CNN images
- show LC subsampling
- show LC merge_close_times (use AGN light curve)
- show LC resampling
- show LC padding
- show LC feature extraction (Kostya's package)

Mention host galaxy + spectroscopy in future
MAKE FIGURE OF CLASSES


